In [1]:
import sys
import importlib

In [2]:
from saxstats import saxstats
from Ligand import Ligand
from WAXS import *
from PDB import PDB
import rdkit
from rdkit import Chem
import numpy as np

In [3]:
pro = PDB('1FIN_apo.pdb')

This PDB does not seem to have hydrogens modeled


In [4]:
print(np.sum([True for x in pro.elements if x == 'H']))
print(np.sum([True for x in pro.elements if x != 'H']))

0.0
2398


In [52]:
#lig = Ligand('c1ccccc1CCC')
lig = Ligand('1FIN_ligand.pdb')
lig.generate_conformers()
Chem.MolToPDBFile(lig.molecule, 'test.pdb')
# print(lig.get_coordinates(0))

Generated 10 conformers


RDKit WARNING: [18:04:21] Molecule does not have explicit Hs. Consider calling AddHs()
[18:04:21] Molecule does not have explicit Hs. Consider calling AddHs()
RDKit WARNING: [18:04:21] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[18:04:21] UFFTYPER: Warning: hybridization set to SP3 for atom 0
[18:04:21] UFFTYPER: Warning: hybridization set to SP3 for atom 4
RDKit WARNING: [18:04:21] UFFTYPER: Warning: hybridization set to SP3 for atom 4
[18:04:21] UFFTYPER: Warning: hybridization set to SP3 for atom 8
RDKit WARNING: [18:04:21] UFFTYPER: Warning: hybridization set to SP3 for atom 8


In [49]:
import time

In [60]:
t0 = time.time()
pv, pg, ps, lv, lg, ls = overlap_grid(pro, lig, 9, grid_spacing=1.0, radius=1.0)
t1 = time.time()
print(t1-t0)
# saxstats.write_mrc(pv, ps, 'test_protein_convolved.mrc')
# saxstats.write_mrc(lv, ls, 'test_ligand.mrc')

[-8. -6. -4.] [7. 4. 4.] [15. 10.  8.]
[-8.59527503e-17  1.36091855e-15  2.68036780e-01] [8.         6.         4.26803678]
[-8. -6. -4.] [8.         6.         4.53607356] [16.         12.          8.53607356]
[-26. -38. -31.] [28. 32. 33.] [54. 70. 64.]
[55 71 65] [17 14 10] (253825, 3) (2398, 3) (2380, 3) (31, 3)
Done with ligand volume
2395
87505
240404
7090
0.18297886848449707


In [7]:
pg[pv.flatten()]

array([[-26.,  -7.,   6.],
       [-26.,  -6.,   6.],
       [-26.,  -6.,   7.],
       ...,
       [ 28.,  14.,   5.],
       [ 28.,  14.,   6.],
       [ 28.,  15.,   5.]])

In [8]:
lig.elements

array(['P', 'O', 'O', 'O', 'P', 'O', 'O', 'O', 'P', 'O', 'O', 'O', 'O',
       'C', 'C', 'O', 'C', 'O', 'C', 'O', 'C', 'N', 'C', 'N', 'C', 'C',
       'N', 'N', 'C', 'N', 'C'], dtype='<U1')

In [9]:
def write_grid_search_xyz(lig, gp, fname='grid_ligand.xyz'):
    lig_coord_init = lig.get_coordinates(0)
    lig_coord_origin = lig_coord_init - np.mean(lig_coord_init, axis=0) #- np.mean(lig_coord_init, axis=0)
    with open(fname, 'w') as f:
        for idx, gp_xyz in enumerate(gp):
            lig_coord_this_grid = lig_coord_origin + gp_xyz
            f.write(f'{lig.num_atoms+1}\n')
            f.write(f'Grid pt {idx}: {gp_xyz[0]:.2f} {gp_xyz[1]:.2f} {gp_xyz[2]:.2f}\n')
            for ele, xyz in zip(lig.elements, lig_coord_this_grid):
                f.write(f'{ele} {xyz[0]} {xyz[1]} {xyz[2]}\n')
            f.write(f'X {gp_xyz[0]} {gp_xyz[1]} {gp_xyz[2]}\n')
        f.write('\n\n')

In [10]:
write_grid_search_xyz(lig, pg[pv.flatten()], fname='grid_ligand3.xyz')

In [11]:
write_grid_search_xyz(lig, [lig.get_coordinates(0).mean(0)], fname='ligand_COM.xyz')

In [12]:
lig.get_coordinates(0).mean(0)

array([ 7.56209677, -9.4603871 , -2.96754839])

In [13]:
np.stack(min_max(lig.get_coordinates(0))).mean(0)

array([ 7.108 , -9.8465, -3.143 ])

In [16]:
pocket = PDB('test_fpocket/1FIN_apo_out/1FIN_apo_out.pdb')

This PDB does not seem to have hydrogens modeled


In [33]:
pocket_coord = pocket.pdb.coords[pocket.elements == 'Ve']

In [34]:
pocket_coord

array([[13.756, -8.239, -7.681],
       [13.826, -8.157, -7.646],
       [13.726, -8.139, -7.627],
       ...,
       [17.881,  8.429, -5.555],
       [17.822,  8.284, -5.585],
       [17.664,  8.338, -4.666]])

In [35]:
pg.shape

(253825, 3)

In [54]:
pv2 = distance_matrix(pg[pv.flatten()], pocket_coord).min(1) < 2

In [48]:
write_grid_points(pg[pv.flatten()], pv2, fname='grid3.pdb')

In [47]:
pg[pv.flatten()][pv2].shape

(99, 3)